In [1]:
import pandas as pd
import numpy as np
import nltk

In [2]:
blog_df = pd.read_csv('blogtext.csv.zip')

In [3]:
blog_df.head()

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...


In [4]:
blog_df = blog_df.drop(['id', 'date'], axis = 1)

In [5]:
blog_df['age'] = blog_df['age'].astype('object')

In [6]:
blog_df.shape

(681284, 5)

In [7]:
blog_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 681284 entries, 0 to 681283
Data columns (total 5 columns):
gender    681284 non-null object
age       681284 non-null object
topic     681284 non-null object
sign      681284 non-null object
text      681284 non-null object
dtypes: object(5)
memory usage: 26.0+ MB


In [8]:
import re
blog_df['Text']=blog_df['text'].apply(lambda x: re.sub(r'[^A-Za-z]+',' ',x))

In [9]:
blog_df['Text']=blog_df['Text'].apply(lambda x: x.lower())

In [10]:
blog_df['Text']=blog_df['Text'].apply(lambda x: x.strip())

In [11]:
from nltk.corpus import stopwords
stopwords=set(stopwords.words('english'))

In [12]:
blog_df['Text']=blog_df['Text'].apply(lambda x: ' '.join([words for words in x.split() if words not in stopwords]))

In [13]:
blog_df['labels']=blog_df.apply(lambda col: [col['gender'],str(col['age']),col['topic'],col['sign']], axis=1)

In [14]:
blog_df=blog_df[['Text','labels']]

In [15]:
blog_df.head()

,Text,labels
0,info found pages mb pdf files wait untill team...,"[male, 15, Student, Leo]"
1,team members drewes van der laag urllink mail ...,"[male, 15, Student, Leo]"
2,het kader van kernfusie op aarde maak je eigen...,"[male, 15, Student, Leo]"
3,testing testing,"[male, 15, Student, Leo]"
4,thanks yahoo toolbar capture urls popups means...,"[male, 33, InvestmentBanking, Aquarius]"


In [16]:
X = blog_df['Text']
y = blog_df['labels']

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

In [18]:
vectorizer=CountVectorizer(ngram_range = (1,2), stop_words=stopwords,min_df = 0.15, max_df = 0.8, max_features = 100)

In [19]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

In [20]:
X_train_data = vectorizer.fit_transform(X_train)
X_test_data = vectorizer.transform(X_test)

In [21]:
X_train_data

<476898x36 sparse matrix of type '<class 'numpy.int64'>'
	with 3707616 stored elements in Compressed Sparse Row format>

In [22]:
X=vectorizer.fit_transform(X)

MemoryError: 

In [ ]:
label_counts=dict()

for labels in blog_df.labels.values:
    for label in labels:
        if label in label_counts:
            label_counts[label]+=1
        else:
            label_counts[label]=1

In [ ]:
label_counts

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
binary = MultiLabelBinarizer(classes=sorted(label_counts.keys()))

In [ ]:
#Converting the train and test labels into binary form
y = binary.fit_transform(blog_df.labels)

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
model=LogisticRegression(solver='lbfgs')

In [ ]:
model=OneVsRestClassifier(model)

In [ ]:
model.fit(X_train,y_train)

In [ ]:
y_predict = model.predict(X_test)

In [ ]:
y_predict_inversed = binarizer.inverse_transform(y_predict)
y_test_inversed = binarizer.inverse_transform(y_test)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score

def print_evaluation_scores(y_test, y_predict):
    print('Accuracy score: ', accuracy_score(y_test, y_predict))
    print('F1 score: ', f1_score(y_test, y_predict, average='micro'))
    print('Average precision score: ', average_precision_score(y_test, y_predict, average='micro'))
    print('Average recall score: ', recall_score(y_test, y_predict, average='micro'))

In [ ]:
print_evaluation_scores(y_test, y_predict)

In [ ]:
for i in range(5):
    print('Text:\t{}\nTrue labels:\t{}\nPredicted labels:\t{}\n\n'.format(
        X_test[i],
        ','.join(y_test_inversed[i]),
        ','.join(y_predict_inversed[i])
    ))